In [1]:
#IMPORTAMOS LIBRERÍA PARA CONECTAR MYSQL-PYTHON
import mysql.connector
#IMPORTAMOS LIBRERÍA PARA INTERACTUAR CON SERVICIOS WEB Y CONSUMIR APIs
import requests
#IMPORTAMOS LIBRERÍA PARA ACCEDER A VARIABLES Y FUNCIONES ESPECÍFICAS DEL INTERPRETE DE PYTHON
import sys
#IMPORTAMOS LIBRERÍA PARA CARGAR Y LEER ARCHIVOS CSV
import csv
##DE LA LIBRERÍA PATHLIB IMPORTAMOS LA FUNCIÓN PATH PARA DEFINIR UNA RUTA HACIA NUESTROS ARCHIVOS
from pathlib import Path

In [65]:
#RUTA HACIA NUESTROS CSV EN EL SERVIDOR DAYHOFF
run_basepath = Path("/home/alumno15/Sistemas Bioinformáticos/Practica2/runs/")

In [2]:
#DEFINIMOS LA CLASE QUE REPRESENTA LA INFORMACIÓN DE UN GEN 
class Gene:
    def __init__(self, gen_symbol, expression):
        #GEN SYMBOL DEL GEN(ES DECIR, ID DE LA BASE DE DATOS SYMBOL --> EJEMPLO: IL6)
        self.gen_symbol = gen_symbol
        #NÚMERO DE LECTURAS DE LOS GENES (EXPRESIÓN)
        self.expression = expression
    
    def __str__(self) :
        return self.gen_symbol
    
    def __repr__(self) :
        return self.gen_symbol 
        
#DEFINIMOS LA CLASE QUE REPRESENTA LA INFORMACIÓN DE UNA MUESTRA
class Sample:
    def __init__(self, sample_id):
        #ID DE LAS MUESTRAS
        self.sample_id = sample_id
        #DICCIONARIO DE GENES ASOCIADOS A CADA MUESTRA
        self.genes = {}
    
    def __str__(self) :
        return self.sample_id
    
    def __repr__(self) :
        return self.sample_id
    
#DEFINIMOS LA CLASE QUE REPRESENTA LA INFORMACIÓN DE UN RUN (PROCESO DE SECUENCIACIÓN)
class Run:
    def __init__(self, run_name):
        #NOMBRE DE CADA RUN 
        self.run_name = run_name
        #DICCIONARIO DE SAMPLES S+NUMBER CONTENIDO EN CADA RUN  
        self.samples = {}
        
    def __str__(self) :
        return self.run_name
    
    def __repr__(self) :
        return self.run_name

In [5]:
#CREAMOS Y DEVOLVEMOS UNA FUNCIÓN DE CONEXIÓN A UNA BASE DE DATOS CON PARÁMETROS ESPECÍFICOS EN MYSQL
def create_connection(host, user, password, database):
    mydb = mysql.connector.connect(
        host=host,
        user=user,#Usuario
        password=password,#contraseña
        database=database #esquema
    )
    return mydb

In [8]:
#GUARDAMOS EN LA VARIABLE CONEXION LA FUNCIÓN PREVIAMENTE CREADA PARA MANEJARLA MÁS FÁCILMENTE
conexion = create_connection("dayhoff.inf.um.es", "alubio08", "alubio08PW-", "alubio08")
print(conexion)

In [7]:
#CERRAMOS CONEXIÓN CUANDO SEA NECESARIO. POR EJEMPLO PARA VOLVER A CARGAR NUESTRO ARCHIVO BASE_DATOS.sql
conexion.close()

In [10]:
#FUNCIÓN QUE CARGARÁ CADA RUN EN UN DICCIONARIO DE DICCIONARIOS DESDE UNA RUTA Y CON UN DELIMITADOR CONCRETO
#BASICAMENTE HAREMOS UN DICCIONARIO RUN CON CLAVE EL NOMBRE DEL RUN Y VALOR EL CONTENIDO DE UN RUN
#EN CADA RUN HABRÁ OTRO DICCIONARIO CON CLAVE(ID DE LAS MUESTRAS) Y VALOR EL CONTENIDO DE LAS MUESTRAS
#EN CADA MUESTRA HABRÁ OTRO DICCIONARIO CON CLAVE (GEN_SYMBOL) Y VALOR EXPRESIÓN(NUMERO DE LECTURAS)
def load_runs(run_basepath, delimitador):
    runs = {}
    index = 1
    run_name = ""
    print("Cargando archivos CSV...")
    #USAMOS RGLOB PARA BÚSQUEDA RECURSIVA EN RUTA DE DIRECTORIOS
    for file in run_basepath.rglob("*.csv"):
        print(f"{file}")
        run_name = f"Run{index}"
        runs[run_name] = Run(run_name)
        index += int(1)
        #SOLO VAMOS A PERMITIR UN TIPO DE DELIMITADOR CONCRETO
        with open(file, mode='r') as csv_file:
            csv_reader = csv.DictReader(csv_file, delimiter=delimitador)
            for row in csv_reader:
                if "gene" not in row:
                    continue
                #POR CADA FILA CREAMOS DICCIONARIO DE PARES MUESTRA-GEN SABIENDO QUE TODAS LAS MUESTRAS COMIENZAN POR S
                for key, expression in row.items():
                    gen = Gene(row["gene"], None)
                    if key.startswith("S"):
                        sample_id = key  #UTILIZAR DIRECTAMENTE LA CLAVE COMO SAMPLE_ID
                        
                    else:
                        continue
                    #VAMOS INCORPORANDO A RUN EL DICCIONARIO DE MUESTRAS
                    if sample_id not in runs[run_name].samples:
                        runs[run_name].samples[sample_id] = Sample(sample_id)
                    gen.expression = expression
                    expression = int(expression)
                    #VAMOS INCORPORANDO A CADA MUESTRA EL DICCIONARIO DE GENES
                    runs[run_name].samples[sample_id].genes[gen.gen_symbol] = gen
    return runs

In [21]:
#GUARDAMOS DICCIONARIOS EN LA VARIABLE ESTRUCTURA_DATOS_RUNS
estructura_datos_runs = load_runs(run_basepath, ";")

Cargando archivos CSV...
/home/alumno15/Sistemas Bioinformáticos/Practica2/runs/run1.csv
/home/alumno15/Sistemas Bioinformáticos/Practica2/runs/run2.csv
/home/alumno15/Sistemas Bioinformáticos/Practica2/runs/run3.csv


In [23]:
#BUCLE ANIDADO PARA VISUALIZAR QUE LA CARGA DE DICCIONARIOS SE REALIZA DE FORMA CORRECTA
for run in estructura_datos_runs:
    for sample in estructura_datos_runs[run].samples:
        #OJO! SE HACE NECESARIA LA EJECUCIÓN DE ESTE BLOQUE PARA QUE FUNCIONEN LOS SIGUIENTES
        estructura_datos_runs[run].samples[sample].genes

In [24]:
#GUARDAMOS EL DICCIONARIO GENES EN LA VARIABLE GENES 
genes = estructura_datos_runs[run].samples[sample].genes

In [28]:
#FUNCIÓN PARA OBTENER EL ID ENSEMBL CORRESPONDIENTE AL SYMBOL DE CADA GEN DE H.SAPIENS
def funcion_sinonimo_ensembl(gen_symbol) :
    #Definimos la URL base del servidor Ensembl REST API
    server = "https://rest.ensembl.org"
    #Construimos la extensión de la URL para la solicitud específica utilizando el symbol del gen como argumento
    ext = "/xrefs/symbol/homo_sapiens/" + gen_symbol + "?"
    #Usamos función GET al servidor Ensembl REST API utilizando la URL construida
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
    #Comprobamos si la respuesta es exitosa, en caso contrario devuelve none
    if not r.ok:
     return None
    #Decodificamos el contenido JSON de la respuesta de la solicitud
    decoded = r.json()
    #Recorremos la lista de resultados decodificados. Si encuentra un identificador que comienza con "ENSG" devuelve ese identificador
    for dec in decoded :
        if dec["id"].startswith("ENSG") :
            return dec["id"]
    return None
#Guardamos contenido en un diccionario de symbol de genes con identificadores Ensembl asociados.
sinonimos_ensembl = {gen: funcion_sinonimo_ensembl(genes[gen].gen_symbol) for gen in genes}

In [26]:
#CONTENIDO DEL DICCIONARIO SINONIMOS_ENSEMBL
print(sinonimos_ensembl)

{'MAPK1': 'ENSG00000100030', 'MAP2K1': 'ENSG00000169032', 'KRAS': 'ENSG00000133703', 'HRAS': 'ENSG00000174775', 'RAF1': 'ENSG00000169397', 'NRAS': 'ENSG00000213281', 'PRKACA': 'ENSG00000072062', 'PRKACG': 'ENSG00000165059', 'PRKACB': 'ENSG00000142875', 'MAP2K2': 'ENSG00000126934', 'PRKCA': 'ENSG00000154229', 'NFKB1': 'ENSG00000109320', 'RELA': 'ENSG00000173039', 'MAPK8': 'ENSG00000107643', 'MAPK9': 'ENSG00000050748', 'MAPK10': 'ENSG00000109339', 'PLCB2': 'ENSG00000137841', 'ADCY5': 'ENSG00000173175', 'PLCB1': 'ENSG00000182621', 'CHUK': 'ENSG00000213341', 'IL6': 'ENSG00000136244', 'IKBKG': 'ENSG00000269335', 'GNAQ': 'ENSG00000156052', 'NFKBIA': 'ENSG00000100906', 'ITPR1': 'ENSG00000150995', 'CDKN1A': 'ENSG00000124762', 'MTOR': 'ENSG00000198793', 'GNAI1': 'ENSG00000127955', 'ITPR2': 'ENSG00000123104', 'IL1B': 'ENSG00000125538', 'RHOA': 'ENSG00000067560', 'FOS': 'ENSG00000170345', 'BCL2': 'ENSG00000171791', 'BAD': 'ENSG00000002330', 'SRC': 'ENSG00000197122', 'PLCG2': 'ENSG00000197943', 'C

In [29]:
#LONGITUD DEL DICCIONARIO SINONIMOS_ENSEMBL 
#COMPROBAMOS QUE TENEMOS LOS 64 GENES PRESENTES EN LOS CSV
len(sinonimos_ensembl)

64

In [60]:
#ASEGURAMOS EN PRIMER LUGAR QUE LAS TABLAS QUE VAMOS A CARGAR POSTERIORMENTE ESTÉN VACÍAS.
print("Vaciando tablas [b3b2_expresion, beb2_muestra_run y beb2_run] antes de insertar...")
#Para poder trabajar desde la base de datos tenemos que crear un cursor a nuestra función conexion
mycursor = conexion.cursor()
#Ejecutamos el comando para borrar tablas en MySQL junto con la tabla deseada
mycursor.execute("DELETE FROM b3b2_expresion")
mycursor.execute("DELETE FROM beb2_muestra_run")
mycursor.execute("DELETE FROM beb2_run")
##Cuando hacemos operaciones de INSERT, UPDATE o DELETE 
#siempre tenemos que hacer un commit para que lo persista en la base de datos
conexion.commit()
#Cada vez que hemos terminado una operación con la base de datos cerramos el cursor
mycursor.close()

Vaciando tablas [b3b2_expresion, beb2_muestra_run y beb2_run] antes de insertar...


True

In [61]:
#INSERCIÓN EN LA TABLA BEB2_RUN EL NOMBRE DE LOS RUN DE NUESTROS CSV GUARDADO EN EL PRIMER DICCIONARIO
#DE LA VARIABLE ESTRUCTURA_DATOS_RUNS
print("Insertando datos en tabla [beb2_run]...")
#Necesitamos el cursor a conexion
mycursor = conexion.cursor()
#Vamos a ir contando las filas insertadas a nuestra tabla en MySQL 
rowcount = 0
for run_name, run_instance in estructura_datos_runs.items():
        #Creamos la plantilla de inserción
        sql_insert_run = "INSERT INTO beb2_run (nombre) VALUES (%s)"
        #Por cada valor que recorremos en el diccionario insertamos el nombre del run en la tabla beb2_run
        mycursor.execute(sql_insert_run, (run_name,))
        #Vamos sumando cada inserción a la tabla en la variable rowcount para contar las filas insertadas
        rowcount = rowcount + 1
#Necesario el commit para que persista
conexion.commit()
print(rowcount, "filas insertadas en [beb2_run]")
#De nuevo cerramos el cursor
mycursor.close()

Insertando datos en tabla [beb2_run]...
3 filas insertadas en [beb2_run]


True

In [62]:
#INSERCIÓN EN LA TABLA BEB2_MUESTRA_RUN LAS MUESTRAS DE NUESTRO DICCIONARIO DE SAMPLES
#JUNTO CON SU RUN CORRESPONDIENTE (EN DOS COLUMNAS DISTINTAS DE LA TABLA)
print("Insertando datos en tabla [beb2_muestra_run]...")
#abrimor cursor 
mycursor = conexion.cursor()
#vamos a contar las filas insertadas 
rowcount = 0
#Rescatamos sample_id y run_name de los diccionarios correspondientes 
for run_name, run in estructura_datos_runs.items():
    for sample_id, sample_instance in run.samples.items():
        #Creamos plantilla de inserción en la tabla correspondiente y los valores que obtendrá
        sql_insert_muestra_run = "INSERT INTO beb2_muestra_run (muestra, run) VALUES (%s, %s)"
        #Ejecutamos cada valor rescatado con el cursor
        mycursor.execute(sql_insert_muestra_run, (sample_id, run_name))
        #Vamos sumando de nuevo cada fila insertada para mostrarlas posterioremente
        rowcount = rowcount + 1
#Necesario el commit para que persista
conexion.commit()
print(rowcount, "filas insertadas en [beb2_muestra_run]")
#Cerramos el cursor
mycursor.close()

Insertando datos en tabla [beb2_muestra_run]...
21 filas insertadas en [beb2_muestra_run]


True

In [63]:
#INSERCIÓN EN LA TABLA B3B2_EXPRESION DE 4 COLUMNAS DONDE VAMOS INCLUIR VALORES PROCEDENTES
#DE TODOS LOS DICCIONARIOS INCLUIDOS EN EL DICCIONARIO DE DICCIONARIOS GUARDADO EN LA VARIABLE
#ESTRUCTURA_DATOS_RUNS
print("Insertando datos en tabla [b3b2_expresion]...")
#Abrimos cursor 
mycursor = conexion.cursor()
#Contamos de nuevo las filas insertadas
rowcount = 0
#Contador para genes actualizados en b3b2_gen como conjunto para seguir solo los genes actualizados
updated_genes = set()
#Rescatamos valores de todos los diccionarios
for run_name, run_instance in estructura_datos_runs.items():
    for sample_id, sample_instance in run_instance.samples.items():
        for gen_symbol, gen_instance in sample_instance.genes.items():
            #Guardamos en la variable ensembl_id el gen_symbol rescatado del diccionario sinonimos_ensembl
            ensembl_id = sinonimos_ensembl.get(gen_symbol)
            if not ensembl_id:
                #Si no tenemos el id de ensembl en ensembl_id lo obtenemos via funcion_sinonimo_ensembl
                ensembl_id = funcion_sinonimo_ensembl(gen_symbol)
            if not ensembl_id:
                #Si seguimos sin encontrar ensembl_id, desestima inserción (debe coincidir con los que tenemos en la tabla b3b2_gen)
                print(f"Error: Ensembl ID no encontrado para gen symbol {gen_symbol}. Desestima inserción.")
                continue

            #Resulta que hay 3 genes que aleatoriamente cuando llamamos al REST API de Ensembl puede coger
            #dos id ensembl distintos -> nosotros en nuestra base de datos solo tenemos uno de ellos y debemos
            #respetar la regla de integridad de la base de datos, por lo que controlaremos que solo coja el id
            #ensembl que nosotros tenemos en nuestra tabla b3b2_gen
            if ensembl_id == 'ENSG00000289694':
                ensembl_id = 'ENSG00000070831'
            elif ensembl_id == 'ENSG00000169397':
                ensembl_id = 'ENSG00000132155'
            elif ensembl_id == 'ENSG00000169710':
                ensembl_id = 'ENSG00000026103'
                
            #creamos la plantilla de actualización de la columna gen_symbol de la tabla b3b2_gen
            #en la misma operación de inserción en la tabla b3b2_expresion
            if gen_symbol is not None:
                sql_update_gene_symbol = "UPDATE b3b2_gen SET gene_symbol = %s WHERE gene_ensembl = %s"
                params_update_gene_symbol = (gen_symbol, ensembl_id)
                mycursor.execute(sql_update_gene_symbol, params_update_gene_symbol)
                #Consulta que vamos a guardar en la variable update_genes donde contamos las filas de la
                #columna gene_symbol de la tabla b3b2_gen una vez actualizada y desestimando valores null.
                sql_count_updated_genes = "SELECT COUNT(DISTINCT gene_symbol) FROM b3b2_gen WHERE gene_symbol IS NOT NULL"
                mycursor.execute(sql_count_updated_genes)
                updated_genes = mycursor.fetchone()[0]

            #Plantilla de inserción en la tabla b3b2_expresion
            sql_insert_expresion = (
                "INSERT INTO b3b2_expresion (muestra, gen, run, expresion) "
                "VALUES (%s, %s, %s, %s) "
                "ON DUPLICATE KEY UPDATE expresion = %s"
            )
            params_insert = (sample_id, ensembl_id, run_name, gen_instance.expression, gen_instance.expression)
            
            
            #Mostramos cada valor insertado en la tabla b3b2_expresion de forma ordenada a modo de control
            print("Sample ID:", sample_id)
            print("Ensembl ID:", ensembl_id)
            print("Run Name:", run_name)
            print("Expression:", gen_instance.expression)
            #Ejecución del cursor de plantilla de inserción y parametros a insertar 
            mycursor.execute(sql_insert_expresion, params_insert)
            rowcount = rowcount + 1
#Necesario commit
conexion.commit()
#Cerramos el cursor
mycursor.close()
#Mostramos el número de filas insertadas y actualizadas en ambas tablas
print(rowcount, " filas insertadas en [b3b2_expresion]")
print(updated_genes, " genes actualizados en [b3b2_gen]")

Insertando datos en tabla [b3b2_expresion]...
Sample ID: S1
Ensembl ID: ENSG00000100030
Run Name: Run1
Expression: 96
Sample ID: S1
Ensembl ID: ENSG00000169032
Run Name: Run1
Expression: 52
Sample ID: S1
Ensembl ID: ENSG00000133703
Run Name: Run1
Expression: 5
Sample ID: S1
Ensembl ID: ENSG00000174775
Run Name: Run1
Expression: 27
Sample ID: S1
Ensembl ID: ENSG00000132155
Run Name: Run1
Expression: 42
Sample ID: S1
Ensembl ID: ENSG00000213281
Run Name: Run1
Expression: 95
Sample ID: S1
Ensembl ID: ENSG00000072062
Run Name: Run1
Expression: 10
Sample ID: S1
Ensembl ID: ENSG00000165059
Run Name: Run1
Expression: 69
Sample ID: S1
Ensembl ID: ENSG00000142875
Run Name: Run1
Expression: 90
Sample ID: S1
Ensembl ID: ENSG00000126934
Run Name: Run1
Expression: 24
Sample ID: S1
Ensembl ID: ENSG00000154229
Run Name: Run1
Expression: 37
Sample ID: S1
Ensembl ID: ENSG00000109320
Run Name: Run1
Expression: 73
Sample ID: S1
Ensembl ID: ENSG00000173039
Run Name: Run1
Expression: 77
Sample ID: S1
Ensem